In [1]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Markdown, display

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# Load gemma tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")

# Load gemma model
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [3]:
def extract_skills(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    skills = [word for word, pos in tagged if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    return skills

def match_skills(user_skills, job_description):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([user_skills, job_description])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

def recommend_job(user_skills):
    matches = [(desc, match_skills(user_skills, desc)) for desc in job_descriptions]
    matches.sort(key=lambda x: x[1], reverse=True)
    return matches[0][0] if matches else None

In [4]:
def generate_resume(user_data):
    prompt = f"Create a professional resume for the following details:\n\n{user_data}"
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**input_ids, max_new_tokens=4096)
    resume = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resume

def generate_cover_letter(user_data, job_description):
    prompt = f"Write a cover letter for the following job description:\n\n{job_description}\n\nAnd the following user data:\n\n{user_data}"
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**input_ids, max_new_tokens=4096)
    cover_letter = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return cover_letter

In [5]:
## trending jobs in the market

job_descriptions = [
    "Software Engineer with experience in Python and Machine Learning.",
    "Data Scientist with strong skills in statistics and R programming.",
    "Frontend Developer with expertise in React and JavaScript.",
    "Backend Developer skilled in server-side languages such as Java, Node.js, and PHP. Experience with database management and RESTful APIs.",
    "DevOps Engineer proficient in CI/CD pipelines, containerization with Docker, and cloud services like AWS or Azure. Strong skills in automation and system monitoring.",
    "UX/UI Designer with expertise in creating intuitive and visually appealing user interfaces. Experience with design tools like Figma, Adobe XD, and a strong understanding of user experience principles.",
    "Product Manager experienced in leading cross-functional teams to deliver software products. Strong skills in market research, product lifecycle management, and agile methodologies.",
    "Cybersecurity Analyst knowledgeable in threat assessment, network security, and incident response. Experience with security tools and frameworks, and a solid understanding of compliance regulations.",
    "Full Stack Developer proficient in both frontend and backend technologies. Experience with frameworks like Angular or Vue.js, and server-side languages such as Python or Ruby.",
    "Mobile App Developer experienced in developing applications for iOS and Android platforms. Proficiency in Swift, Kotlin, or React Native, and a strong understanding of mobile development best practices.",
    "Database Administrator skilled in managing and optimizing databases such as MySQL, PostgreSQL, or MongoDB. Experience with data backup, recovery, and performance tuning.",
    "Cloud Architect with expertise in designing and implementing cloud solutions. Proficient with cloud platforms like AWS, Google Cloud, or Azure, and strong skills in cloud security and cost management.",
    "Business Intelligence Analyst proficient in data analysis and visualization tools such as Tableau or Power BI. Strong skills in SQL and experience with data warehousing solutions.",
]

In [6]:
def get_user_data():
    name = input("Enter your name: ")
    email = input("Enter your email: ")
    experience = input("Enter your experience: ")
    skills = input("Enter your skills (comma-separated): ")
    education = input("Enter your education: ")

    user_data = {
        'name': name,
        'email': email,
        'experience': experience,
        'skills': skills,
        'education': education
    }

    return user_data

In [7]:
# Get user data
user_data = get_user_data()

Enter your name: Samriddha Sil
Enter your email: samriddha.sil297@gmail.com
Enter your experience: Don't have any experience, I am a fresher
Enter your skills (comma-separated): Cpp, Java, Python, Natural Language Processing, Machine Learning, Problem Solving Abilities, Time Management
Enter your education: B.Tech. in Computer Science and Engineering


In [8]:
# Extract skills
user_skills = extract_skills(user_data['skills'])
print(user_skills)

['Cpp', 'Java', 'Python', 'Natural', 'Language', 'Processing', 'Machine', 'Learning', 'Problem', 'Solving', 'Abilities', 'Time', 'Management']


In [9]:
# Recommend a job description
recommended_job = recommend_job(" ".join(user_skills))
print(recommended_job)

Software Engineer with experience in Python and Machine Learning.


In [10]:
resume = generate_resume(user_data)
display(Markdown(resume))

Create a professional resume for the following details:

{'name': 'Samriddha Sil', 'email': 'samriddha.sil297@gmail.com', 'experience': "Don't have any experience, I am a fresher", 'skills': 'Cpp, Java, Python, Natural Language Processing, Machine Learning, Problem Solving Abilities, Time Management', 'education': 'B.Tech. in Computer Science and Engineering'}

**Samriddha Sil**
samriddha.sil297@gmail.com
[City, State, Zip]
[Phone Number]
[LinkedIn Profile URL]

**Summary**

Highly motivated and eager to learn and grow in the field of computer science. Seeking a challenging role where I can utilize my skills and knowledge to make a significant contribution to an organization.

**Skills**

* C++
* Java
* Python
* Natural Language Processing
* Machine Learning
* Problem Solving Abilities
* Time Management

**Education**

* B.Tech. in Computer Science and Engineering

**Additional Information**

* Passionate about technology and its potential to solve real-world problems.
* Strong analytical and problem-solving skills.
* Excellent communication and interpersonal skills.

**References**

Available upon request.

In [11]:
cover_letter = generate_cover_letter(user_data, recommended_job)
display(Markdown(cover_letter))

Write a cover letter for the following job description:

Software Engineer with experience in Python and Machine Learning.

And the following user data:

{'name': 'Samriddha Sil', 'email': 'samriddha.sil297@gmail.com', 'experience': "Don't have any experience, I am a fresher", 'skills': 'Cpp, Java, Python, Natural Language Processing, Machine Learning, Problem Solving Abilities, Time Management', 'education': 'B.Tech. in Computer Science and Engineering'}

**[Your Name]**
**[Your Address]**
**[City, State, Zip]**
**[Email Address]**
**[Phone Number]**
**[Date]**

**[Hiring Manager Name]**
**[Company Name]**
**[Company Address]**

**Subject: Application for Software Engineer Position**

Dear [Hiring Manager Name],

I am writing to express my keen interest in the Software Engineer position at [Company Name], as advertised on [Platform]. With my strong foundation in Python and Machine Learning, I am confident that I possess the skills and experience necessary to make a significant contribution to your esteemed organization.

While I may lack formal experience in the software development industry, I am a highly motivated and self-taught individual with a relentless passion for technology. My educational background in Computer Science and Engineering has equipped me with a solid understanding of the fundamentals of programming, data structures, algorithms, and object-oriented programming.

My key strengths include:

- **Exceptional programming skills:** I am proficient in Python, enabling me to build efficient and scalable web applications.
- **Machine Learning expertise:** I am passionate about leveraging machine learning techniques to solve complex problems and improve user experiences.
- **Problem-solving abilities:** I am adept at identifying and resolving technical challenges efficiently.
- **Time management:** I possess strong time management skills, enabling me to prioritize tasks and meet deadlines consistently.

I am eager to join a dynamic team of professionals and contribute my expertise to your innovative projects. I am confident that my skills and experience align perfectly with the requirements of this position, and I am eager to learn and grow within your organization.

I have attached my resume for your review, which provides further details of my qualifications and experience. I am available for an interview at your earliest convenience and would appreciate the opportunity to discuss how my skills can benefit your company.

Thank you for your time and consideration.

Sincerely,
[Your Name]